# **Ini Untuk EL SETUP**
run cellnya aja

In [1]:
import numpy as np
import os
import cv2 as cv
import mediapipe as mp
from mediapipe.python.solutions.pose import PoseLandmark
import math

# *Sesuain sama kelas yang mau dipake misalnya kursi roda wajib 5, soalnya kontrolnya 5 doang*

In [2]:
# START COLLECT DATASET

# DATA_PATH = os.path.join('My_Datasets')
# DATA_PATH = os.path.join('My_Temp')

# actions = np.array(['Berapa'])
# no_sequences = 10

# temp_no_sequences = 10
# no_sequences = temp_no_sequences
# sequence_length = 30

# END COLLECT DATAASET

    
# # START NORMALIZING DATASET
DATA_PATH = os.path.join('El_Agung') #ganti ini bos, ini nama foldernya

actions = np.array(["Kanan", "Maju", "Stop", "Mundur", "Kiri" #kalau kursi roda ini udah fix, kalau yang lain sesuain
                    ])
no_sequences = 30
sequence_length = 30 #ini eksperiment aja kalau modellu jelek pake setting ini bisa ganti, boleh lebih banyak inget tergantung epoch dkk juga
# # END NORMALIZING DATASET

In [3]:
def create_folder(actions, data_path, no_sequences):
    for action in actions: 
        for sequence in range(no_sequences):
            try: 
                os.makedirs(os.path.join(data_path, action, str(sequence)))
            except:
                pass



# **Kalau mau modif dataset, kalau belum punya JANGAN RUN INI**


In [ ]:
def change_file_sequence(folder_name, total_sequences, start_next_sequences, actions):
    DATA_PATH_CHANGE = os.path.join(rf'C:\Users\krisn\OneDrive\Desktop\Learning\machine-learning-study\lstm-bisindo-translator\{folder_name}')

    next_folder_number = start_next_sequences

    for action in actions:
        for new_sequence in range(total_sequences):
            PATH_OLD = os.path.join(DATA_PATH_CHANGE, action, str(new_sequence))
            PATH_NEW = os.path.join(DATA_PATH_CHANGE, action, str(new_sequence + next_folder_number))

            os.rename(PATH_OLD, PATH_NEW)         

In [30]:
change_file_sequence(DATA_PATH, total_sequences=10 - 0, start_next_sequences=20, actions=actions)

# **lanjut kesini dlu**

In [4]:

create_folder(actions, DATA_PATH, no_sequences)

# **Collect Keypoint Values for Training and Testing**

In [5]:
#define mediapipe holistic and drawing utils
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [6]:
#detect using mediapipe model
def media_pipe_detection(image, model):
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB) 
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv.cvtColor(image, cv.COLOR_RGB2BGR) 
    return image, results

def draw_land_marks(image, results):
    custom_pose_connections = list(mp_pose.POSE_CONNECTIONS)
    
    excluded_landmarks = [ 
        PoseLandmark.NOSE,
        PoseLandmark.LEFT_EYE_INNER,
        PoseLandmark.LEFT_EYE,
        PoseLandmark.LEFT_EYE_OUTER,
        PoseLandmark.RIGHT_EYE_INNER,
        PoseLandmark.RIGHT_EYE,
        PoseLandmark.RIGHT_EYE_OUTER,
        PoseLandmark.LEFT_EAR,
        PoseLandmark.RIGHT_EAR,
        PoseLandmark.MOUTH_LEFT,
        PoseLandmark.MOUTH_RIGHT,
        PoseLandmark.LEFT_HIP,
        PoseLandmark.RIGHT_HIP,
        PoseLandmark.LEFT_KNEE,
        PoseLandmark.RIGHT_KNEE,
        PoseLandmark.LEFT_ANKLE,
        PoseLandmark.RIGHT_ANKLE,
        PoseLandmark.LEFT_HEEL,
        PoseLandmark.RIGHT_HEEL,
        PoseLandmark.LEFT_FOOT_INDEX,
        PoseLandmark.RIGHT_FOOT_INDEX
    ]

    for landmark in excluded_landmarks:
        custom_pose_connections = [connection_tuple for connection_tuple in custom_pose_connections if landmark.value not in connection_tuple]

    mp_drawing.draw_landmarks(image, results.pose_landmarks, connections=custom_pose_connections)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

def extract_keypoints(results):
    if results.pose_landmarks:
        selected_pose_landmarks = results.pose_landmarks.landmark[11:23]
        pose = np.array([[res.x, res.y] for res in selected_pose_landmarks]).flatten()
    else:
        pose = np.zeros(22*2)

    left_hand = np.array([[res.x, res.y] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*2)
    right_hand = np.array([[res.x, res.y] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*2)
   
    return np.concatenate([pose, left_hand, right_hand])

# *cap=cv.videocapture(0) untuk kamera utama, misalnya yang muncul droidcam atau obs, ganti ke 1 /2/3/ dst*

In [7]:
cap = cv.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):

                ret, frame = cap.read()

                image, results = media_pipe_detection(frame, holistic)
                
                #get ready for capture current sequence
                if frame_num == 0: 
                    cv.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv.LINE_AA)
                    cv.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv.LINE_AA)
                    cv.imshow('OpenCV Feed', image)
                    cv.waitKey(5000)
                
                #prepare for next sequence
                elif frame_num == sequence_length - 1 :
                    cv.putText(image, 'NEXT SEQUENCE', (120,200), 
                               cv.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv.LINE_AA)
                    cv.imshow('OpenCV Feed', image)
                    cv.waitKey(5000)

                else: 
                    cv.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv.LINE_AA)
                    cv.imshow('OpenCV Feed', image)

                black_bg = np.zeros((frame.shape[0], frame.shape[1], frame.shape[2]))

                draw_land_marks(image, results)
                draw_land_marks(black_bg, results)

                cv.imwrite(os.path.join(DATA_PATH, action, str(sequence), f"{frame_num}-clear.jpg"), frame)
                cv.imwrite(os.path.join(DATA_PATH, action, str(sequence), f"{frame_num}.jpg"), image)
                cv.imwrite(os.path.join(DATA_PATH, action, str(sequence), f"{frame_num}-black.jpg"), black_bg)
                
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                if cv.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv.destroyAllWindows()

d:\Castano\sign-language-lstm-master\.venv\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


NameError: name 'cap' is not defined

# **Normalize Datasets**

In [8]:
coor_x = []
coor_y = []
coordinates = []

for action in actions:
     for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            for index, coor in enumerate(res):
                coordinates.append(coor)

                if(index % 2 != 0):
                    coor_x.append(coor)
                else:
                    coor_y.append(coor)

In [9]:
print(len(coordinates))

486000


In [10]:
midpoint_x = (coor_x[0] + coor_x[1])/2
midpoint_y = (coor_y[0] + coor_y[1])/2

# delta_x = midpoint_x - frame_width/2
# delta_y = midpoint_y - frame_height/2

shoulder_delta_x = coor_x[1] - coor_x[0]
shoulder_delta_y = coor_y[1] - coor_y[0]
shoulder_width = math.sqrt(pow(shoulder_delta_x, 2) + pow(shoulder_delta_y, 2))

In [11]:
def normalization(midpoint_x, midpoint_y, shoulder_width, coordinates):
    print("Koordinat awal:")
    print("Max: " + str(max(coordinates)))
    print("Min: " + str(min(coordinates)))


    for index, coor in enumerate(coordinates):
        
        #position invariant
        if(index % 2 != 0):
            coordinates[index] -= midpoint_x
        else:
            coordinates[index] -= midpoint_y

        #scale invariant
        coordinates[index] /= shoulder_width

    print("\nKoordinat normalisasi:")
    print("Max: " + str(max(coordinates)))
    print("Min: " + str(min(coordinates)))
    return coordinates

def save_normalization(paths, actions, no_sequences, sequence_length, coor_norm):
    total_landmarks = 108
    temp_index = 0

    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):
                saved_coor_norm = coor_norm[temp_index:temp_index+total_landmarks]

                norm_npy_path = os.path.join(paths, action, str(sequence), str(frame_num) + "-norm")
                np.save(norm_npy_path, saved_coor_norm)

                temp_index += total_landmarks

In [12]:
normalized_coor = normalization(midpoint_x, midpoint_y, shoulder_width, coordinates)

save_normalization(DATA_PATH, actions, no_sequences, sequence_length, normalized_coor)

Koordinat awal:
Max: 1.7830085754394531
Min: -0.0710531696677208

Koordinat normalisasi:
Max: 2.2158788761820043
Min: -2.4366689431895447


In [13]:
check_coordinates = []

for action in actions:
     for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}-norm.npy".format(frame_num)))
            for index, coor in enumerate(res):
                check_coordinates.append(coor)

In [14]:
print(len(check_coordinates))
print("Max: " + str(max(check_coordinates)))
print("Min: " + str(min(check_coordinates)))

486000
Max: 2.2158788761820043
Min: -2.4366689431895447


In [15]:
print(len(coor_x) * 2)
print(coor_x[0])
print(2*30*30*108)
print(12*2 + 21 * 2 * 2)

486000
0.9495095014572144
194400
108


In [16]:
len(actions)

5

In [17]:
108*30*30*14

1360800